## Polynomial Chaos Expansion Example 4

Author: Katiana Kontolati \
Date: December 8, 2020

In this example, PCE is used to generate a surrogate model for a given set of 3D data.

### Dette & Pepelyshev exponential function

$$ f(x) = 100(\exp{(-2/x_1^{1.75})} + \exp{(-2/x_2^{1.5})} + \exp{(-2/x_3^{1.25})})$$

**Description:**  Dimensions: 3

**Input Domain:**  This function is evaluated on the hypercube $x_i \in [0,1]$ for all i = 1,2,3.

**Reference:**  Dette, H., & Pepelyshev, A. (2010). Generalized Latin hypercube design for computer experiments. Technometrics, 52(4).

Import necessary libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from UQpy.distributions import Uniform, JointIndependent
from UQpy.surrogates import Polynomials, PolynomialChaosExpansion, LassoRegression, RidgeRegression, \
    LeastSquareRegression, ErrorEstimation, MomentEstimation

Define the function.

In [ ]:
def function(x):
    return 100*(np.exp(-2/(x[:,0]**1.75)) + np.exp(-2/(x[:,1]**1.5)) + np.exp(-2/(x[:,2]**1.25)))

Create a distribution object, generate samples and evaluate the function at the samples.

In [ ]:
np.random.seed(1)

dist = Uniform(loc=0, scale=1)

marg = [dist] * 3
joint = JointIndependent(marginals=marg)

n_samples = 165
x = joint.rvs(n_samples)
y = function(x)

Create an object from the PCE class.

In [ ]:
max_degree = 5
polys = Polynomials(distributions=joint, degree=max_degree)

Compute PCE coefficients using least squares regression, Lasso regression and Ridge regression.

In [ ]:
lstsq = LeastSquareRegression(polynomials=polys)
pce = PolynomialChaosExpansion(regression_method=lstsq)
pce.fit(x, y)

lasso = LassoRegression(polynomials=polys, learning_rate=0.01, iterations=1000, penalty=0.1)
pce2 = PolynomialChaosExpansion(regression_method=lasso)
pce2.fit(x, y)

ridge = RidgeRegression(polynomials=polys, learning_rate=0.01, iterations=1000, penalty=0.1)
pce3 = PolynomialChaosExpansion(regression_method=ridge)
pce3.fit(x, y)

In [ ]:
n_samples = 500
x_val = joint.rvs(n_samples)
y_val = function(x_val)

### Error Estimation

Validation error.

In [ ]:
n_samples = 500
x_val = joint.rvs(n_samples)
y_val = function(x_val)

error = ErrorEstimation(pce_surrogate=pce)
error2 = ErrorEstimation(pce_surrogate=pce2)
error3 = ErrorEstimation(pce_surrogate=pce3)
print('Error from least squares regression is: ', error.validation(x_val, y_val))
print('Error from LASSO is: ', error2.validation(x_val, y_val))
print('Error from Ridge is: ', error3.validation(x_val, y_val))

### Moment Estimation

Returns mean and variance of the PCE surrogate.

In [ ]:
n_mc = 1000000
x_mc = joint.rvs(n_mc)
y_mc = function(x_mc)
mu = np.mean(y_mc)

print('Moments from least squares regression :', MomentEstimation(pce_surrogate=pce).get())
print('Moments from LASSO regression :', MomentEstimation(pce_surrogate=pce2).get())
print('Moments from Ridge regression :', MomentEstimation(pce_surrogate=pce3).get())
print('Moments from MC integration: ',
      (round((1 / n_mc) * np.sum(y_mc), 6), round((1 / n_mc) * np.sum((y_mc - mu) ** 2), 6)))